# pipe

> Common operations around the core datastructures for running a sim

In [ ]:
#| default_exp pipe

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import jax
import jax.numpy as jnp
from jaxtyping import Array, Float
from typing import Callable, Union

from chewc.population import Population
from chewc.sp import SimParam
from chewc.phenotype import set_pheno
from chewc.cross import make_cross

def select_ind(
    pop: Population,
    n_ind: int,
    use: Union[str, Callable[[Population], jnp.ndarray]] = "pheno",
    select_top: bool = True
) -> Population:
    """
    Selects the top or bottom individuals from a population.

    Args:
        pop: The population to select from.
        n_ind: The number of individuals to select.
        use: The criterion for selection. Can be a string ('gv', 'bv', 
             'pheno', 'ebv') or a custom function that takes a 
             Population and returns a 1D array of selection values.
        select_top: If True, selects the highest values; otherwise, 
                    selects the lowest.

    Returns:
        A new, smaller Population object with the selected individuals.
    """
    if isinstance(use, str):
        selection_values = getattr(pop, use)
        if selection_values.ndim > 1:
            # Default to the first trait if multiple exist
            selection_values = selection_values[:, 0]
    else:
        selection_values = use(pop)
    
    # Use JAX's efficient top-k selection for performance
    if select_top:
        # jax.lax.top_k returns values and indices. We need to flip the
        # sign to get the top values, then flip it back.
        _, indices = jax.lax.top_k(selection_values, k=n_ind)
    else:
        _, indices = jax.lax.top_k(-selection_values, k=n_ind)

    # Use jax.tree_util.tree_map to apply the slicing to all array 
    # attributes of the Population object. This is a robust, functional
    # approach that avoids manual slicing of each attribute.
    selected_pop = jax.tree_util.tree_map(lambda x: x[indices], pop)
    
    return selected_pop


def update_pop_values(
    key: jax.random.PRNGKey, 
    pop: Population, 
    sp: SimParam, 
    h2: Float[Array, "nTraits"]
) -> Population:
    """
    Calculates genetic and phenotypic values for a new population.

    This is a common pipeline step after creating progeny, who are born
    with only genotypic information.

    Args:
        key: A JAX random key for phenotypic noise generation.
        pop: The population of new individuals (e.g., progeny).
        sp: The simulation parameters, containing trait information.
        h2: The heritability for the trait(s) being phenotyped.

    Returns:
        A new Population object with updated bv and pheno attributes.
    """
    # Use the existing JIT-compatible function from the pheno module
    # This will calculate gv (and thus bv) and then the phenotype.
    pop_with_values = set_pheno(key, pop, sp.traits, sp.ploidy, h2=h2)
    return pop_with_values

def select_and_cross(
    key: jax.random.PRNGKey,
    pop: Population,
    sp: SimParam,
    n_parents: int,
    n_crosses: int,
    use: str = "pheno"
) -> Population:
    """Selects parents and performs random crosses to create a new generation."""
    key_select, key_cross = jax.random.split(key)
    
    # 1. Select the best individuals to form the parent pool (sexes ignored)
    parent_pool = select_ind(pop, n_parents, use=use)
    
    # 2. Generate a random cross plan from the selected parent pool
    # Any parent can be a mother or a father
    mother_iids = jax.random.choice(key_cross, parent_pool.iid, shape=(n_crosses,))
    key_cross, _ = jax.random.split(key_cross) # Split key for next choice
    father_iids = jax.random.choice(key_cross, parent_pool.iid, shape=(n_crosses,))
    
    cross_plan = jnp.stack([mother_iids, father_iids], axis=1)

    # 3. Create progeny using the existing low-level function
    progeny = make_cross(key_cross, pop, cross_plan, sp)
    
    return progeny

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()